In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from glob import glob
 
from sklearn.model_selection import train_test_split
from sklearn import metrics
 
import cv2
import gc
import os 

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import warnings
warnings.filterwarnings('ignore')


In [ ]:
#pip install opencv-python

In [ ]:

#from zipfile import ZipFile
 
#data_path = 'archive.zip'
 
#with ZipFile(data_path,'r') as zip:
 # zip.extractall()
  #print('The data set has been extracted.')


In [ ]:
dataset_path = '/kaggle/input/lung-and-colon-cancer-histopathological-images/lung_colon_image_set/lung_image_sets/'
print(os.listdir(dataset_path))

In [ ]:
classes = os.listdir(dataset_path)
classes

In [ ]:
IMG_SIZE = 400
SPLIT = 0.2
EPOCHS = 10
BATCH_SIZE = 64
learning_rate = 0.001
num_epochs = 20
NODE = np.int_(IMG_SIZE / 4)

In [ ]:
from torchvision import datasets
from torch.utils.data import DataLoader

# Define transformations (e.g., resizing, normalization)
transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),  # Resize images to 224x224
    transforms.ToTensor(),  # Convert PIL image to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize
])

# Load dataset from local directory
data_dir = "../../lung_colon_image_set/lung_image_sets/"
dataset = datasets.ImageFolder(root=dataset_path, transform=transform)

# Create DataLoader
#dataloader = DataLoader(dataset, batch_size=32, shuffle=True)
# Load dataset


generator1 = torch.Generator().manual_seed(42)
train_set, val_set = torch.utils.data.random_split(dataset, [0.8, 0.2], generator=generator1 )

# Create DataLoader
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True, num_workers=8, pin_memory=True)
test_loader = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=True,)

print(len(train_set))
img, label = dataset[10]
print(img.shape)


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# Creating a CNN class
class Net(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
    self.act1 = nn.Tanh()
    self.pool1 = nn.MaxPool2d(2)
    self.conv2 = nn.Conv2d(32, 8, kernel_size=3, padding=1)
    self.act2 = nn.Tanh()
    self.pool2 = nn.MaxPool2d(2)
    self.fc1 = nn.Linear(8 * NODE * NODE, 64) # NODE is (IMG_SIZE)devided by (2*2) from the max pool.
    self.act3 = nn.ReLU()
    self.fc3 = nn.Linear(64, 64)
    self.fc4 = nn.Linear(64, 32)
    self.fc2 = nn.Linear(32, 3)
  def forward(self, x):
    out = self.pool1(self.act3(self.conv1(x)))
    out = self.pool2(self.act3(self.conv2(out)))
    out = out.view(out.size(0), -1)
    out = self.act3(self.fc1(out))
    out = self.act3(self.fc3(out))
    out = self.act3(self.fc4(out))
    out = self.fc2(out)
    return out


In [ ]:
model = Net()
model = nn.DataParallel(model)
model = model.to(device=device)

# Set Loss function with criterion
loss_fn = nn.CrossEntropyLoss()

# Set optimizer with optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay = 0.01, momentum = 0.5)  

total_step = len(train_loader)

In [ ]:
def training_loop(n_epochs, optimizer, model, loss_fn, train_loader):
   for epoch in range(1, n_epochs + 1):
      loss_train = 0.0
      for imgs, labels in train_loader:
            imgs = imgs.to(device=device)
            labels = labels.to(device=device)
            #model = model.to(device=device)
            outputs = model(imgs)
            loss = loss_fn(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            loss_train += loss.item()


      if epoch == 1 or epoch  != 0:
         print('{} Epoch {}, Training loss {}'.format(
             datetime.datetime.now(), epoch,
             loss_train / len(train_loader)))


            
            


In [ ]:
import datetime
training_loop(n_epochs = 30, optimizer = optimizer, model = model, loss_fn = loss_fn, train_loader = train_loader,)


 

In [ ]:
def validate(model, train_loader, val_loader):
     for name, loader in [("train", train_loader), ("val", val_loader)]:
             correct = 0
             total = 0
             with torch.no_grad():
                for imgs, labels in loader:
                    imgs = imgs.to(device=device)
                    labels = labels.to(device=device)
                    model = model.to(device=device)
                    outputs = model(imgs)
                    _, predicted = torch.max(outputs, dim=1)
                    total += labels.shape[0]
                    correct += int((predicted == labels).sum())

             print("Accuracy {}: {:.2f}".format(name , correct / total))

   

In [ ]:
validate(model, train_loader, test_loader)


In [ ]:
!lscpu

In [ ]:
#torch.save(model.state_dict(), data_dir + 'Lung_cancer.pt')
